In [1]:
using BenchmarkTools
using LinearAlgebra
using Random
using Profile

In [2]:
mutable struct LTMADS{T}
    b::Dict{T,Vector{T}}
    i::Dict{T,Int64}
    maximal_basis::Bool
    LTMADS(;kwargs...) = LTMADS{Float64}(;kwargs...)
    function LTMADS{T}(;maximal_basis=false) where T
        g = new()
        g.b = Dict{T, Vector{T}}()
        g.i = Dict{T, Int64}()
        g.maximal_basis = maximal_basis
        return g
    end
end

In [3]:
mutable struct Mesh{T} <: AbstractMesh 
    G::Matrix{T}
    D::Matrix{T}
    l::Int
    Δᵐ::T
    Δᵖ::T

    # Override constructor for different default meshes for 
    # different poll techniques.
    Mesh(N::Int64) = Mesh{Float64}(N)
    function Mesh{T}(N::Int64) where T
        mesh = new()
        mesh.l = 0
        mesh.Δᵐ = min(1, 4.0^(-mesh.l))
        mesh.Δᵖ = 2.0^(-mesh.l)
        mesh.G = Matrix(I,N,N)
        mesh.D = hcat(Matrix(I,N,N),-Matrix(I,N,N))
        return mesh
    end 
end

In [46]:
setup(;N=4, max=false) = (N, Mesh(N), LTMADS(maximal_basis=max))

@benchmark GenerateDirections(N, m, DG) setup=((N,m,DG)=setup(N=100,max=false))

BenchmarkTools.Trial: 
  memory estimate:  744.77 KiB
  allocs estimate:  438
  --------------
  minimum time:     229.027 μs (0.00% GC)
  median time:      263.759 μs (0.00% GC)
  mean time:        317.758 μs (10.57% GC)
  maximum time:     2.497 ms (80.14% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [52]:
N,m,DG =setup(N=100,max=false)
#GenerateDirections(N, m, DG);
B = LT_basis_generation(m,N,DG)
@code_warntype form_basis_matrix(N, B, DG)

Variables
  #self#::Core.Compiler.Const(form_basis_matrix, false)
  N::Int64
  B::Array{Int64,2}
  DG::LTMADS{Float64}
  d::Array{Float64,1}
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Array{Int64,2}
1 ─       Core.NewvarNode(:(d))
│         Core.NewvarNode(:(@_6))
│   %3  = Core.apply_type(Main.Matrix, Main.Int)::Core.Compiler.Const(Array{Int64,2}, false)
│   %4  = Base.getproperty(DG, :maximal_basis)::Bool
└──       goto #4 if not %4
2 ─ %6  = -B::Array{Int64,2}
│   %7  = Main.hcat(B, %6)::Array{Int64,2}
│   %8  = Base.convert(%3, %7)::Array{Int64,2}
│   %9  = Core.typeassert(%8, %3)::Array{Int64,2}
└──       return %9
3 ─       Core.Compiler.Const(:(goto %12), false)
4 ┄       (d = Main.ones($(Expr(:static_parameter, 1)), N))
│   %13 = (1:N)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_6 = Base.iterate(%13))
│   %15 = (@_6 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #7 if not %16
5 ┄ %1

In [38]:
function GenerateDirections(N::Int64, m::Mesh, DG::LTMADS{T})::Matrix{T} where T
    B = LT_basis_generation(m, N, DG)
    Dₖ = form_basis_matrix(N, B, DG)

    return Dₖ
end

GenerateDirections (generic function with 1 method)

In [53]:
function form_basis_matrix(N::Int64, B::Matrix{Int}, DG::LTMADS{T})::Matrix{Int} where T
    DG.maximal_basis && return [B -B]
    
    
    d = ones(T, N)
    for i in 1:N
        d[i] = -sum(B[i,:])
    end
    
    return [B d]
end

form_basis_matrix (generic function with 1 method)

In [40]:
function LT_basis_generation(m::Mesh, N::Int, DG::LTMADS{T})::Matrix{Int} where T
    b, i = b_l_generation(DG.b, DG.i, m.l, N)
    
    L = L_generation(N, m.l)
    
    B = B_generation(N, i, b, L)
    
    B′ = B′_generation(B, N)
    
    return B′
end

LT_basis_generation (generic function with 1 method)

In [41]:
function B′_generation(B, N; perm=shuffle(1:N))::Matrix{Int}
    B′ = zeros(N,N)
    for (i,e) in enumerate(eachcol(B))
        B′[:,perm[i]] = e
    end
    return B′
end

B′_generation (generic function with 1 method)

In [42]:
function b_l_generation(b::Dict{T,Vector{T}}, i::Dict{T,Int64}, l::Int64, N::Int64)::Tuple{Vector{Int},Int} where T
    if !haskey(b, l)
        i[l] = rand(1:N)
        b[l] = zeros(T, N)
        
        for j in 1:N
            if j == i[l]
                b[l][j] = rand([-2^l, 2^l])
            else
                b[l][j] = rand(-2^l+1:2^l-1)
            end
        end
    end
    return b[l], i[l]
end

b_l_generation (generic function with 1 method)

In [43]:
function L_generation(N::Int64, l::Int64)
    L = zeros(Int64, N-1,N-1)

    for i=1:N-1, j=1:N-1
        if j==i
            L[i,j] = rand([2^l, -2^l])
        elseif j < i
            L[i,j] = rand(1-2^l:-1+2^l)
        end
    end
    
    return L
end

L_generation (generic function with 1 method)

In [44]:
function B_generation(N::Int64, i::Int64, b::Vector{Int64}, L::Matrix{Int64}; perm=shuffle(setdiff(1:N, i)))
    B = zeros(Int64, N,N-1)
    for (i,e) in enumerate(eachrow(L))
        B[perm[i],:] = e
    end
    B = [B b]
    
    return B
end

B_generation (generic function with 2 methods)